<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab05/expansao_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [249]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import heapq
import time
import bisect
import random
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_lab_01/master/output/results.csv') 
                                            # Resultados adquiridos do site brasil-247 em abril de 2019

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Para a realização dessa tarefa, fizemos o ranqueamento das palavras presentes nos documentos adquiridos do site https://www.brasil247.com/ em Abril de 2019. Criando um dicionário que irá relacionar as palavras com suas listas invertidas com os documentos e os scores da palavra.

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

Abaixo, temos as palavras que terão sua consulta expandida

In [0]:
queries = ["bolsonaro", "educação", "ministério", "brasil", "economia"]

### 1. Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

Definimos então, os valores que serão necessários para o cálculo das 4 métricas, sendo eles:
  * O número total de documentos no ranking;
  * O número de documentos em que uma palavra aparece;
  * O número de documentos em que ambas as palavras aparecem.

In [0]:
def n_docs_word(word, ranking):
  return len(ranking[word])

def n_docs_both(a, b, ranking):
  docs_a = [doc for doc,score in ranking[a]]
  docs_b = [doc for doc,score in ranking[b]]
  n = 0
  
  for doc in docs_a:
    if doc in docs_b:
      n += 1
      
  return n

Aqui, temos o cálculos das métricas dispostas no livro:
  * MIM;
  * EMIM;
  * X²;
  * Dice.
  
Que irão indicar a proximidade das duas palavras dentro do ranking.

In [0]:
def calc_mim(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  mim = n_ab / quo if quo != 0 else 0
  return (mim,b)
  
def calc_emim(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  exp = n * (n_ab / quo) if quo != 0 else 0
  emim = n_ab * np.log10(exp) if exp != 0 else 0
  return (emim, b)
  
def calc_x(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  x = (n_ab - (1 / n) * n_a * n_b) ** 2 / quo if quo != 0 else 0
  return (x,b)
  
def calc_dice(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  dice = n_ab / (n_a + n_b)
  return (dice,b)

Nesse momento, iremos fazer a criação da tabela com as top-10 palavras que se aproximam da consulta.

In [0]:
def create_metrics_table(query, ranking, n):
  top_10_table = pd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for word in ranking:
    if word != query:
      bisect.insort(MIM, calc_mim(query, word, ranking))
      bisect.insort(EMIM,calc_emim(query, word, ranking, n))
      bisect.insort(X,calc_x(query, word, ranking, n))
      bisect.insort(Dice,calc_dice(query, word, ranking))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in MIM[:10]]
  top_10_table['EMIM'] = [word[1] for word in EMIM[:10]]
  top_10_table['X²'] = [word[1] for word in X[:10]]
  top_10_table['Dice'] = [word[1] for word in Dice[:10]]
  
  top_10_table.index += 1
  
  return top_10_table

In [0]:
n = result.text.count()
table_1 = create_metrics_table(queries[0], indexes, n)
table_2 = create_metrics_table(queries[1], indexes, n)
table_3 = create_metrics_table(queries[2], indexes, n)
table_4 = create_metrics_table(queries[3], indexes, n)
table_5 = create_metrics_table(queries[4], indexes, n)

####1.1 Top-10 palavras associadas a 'Bolsonaro' 

In [256]:
table_1

,MIM,EMIM,X²,Dice
1,única,jair,jair,jair
2,últimas,presidente,psl,presidente
3,ódio,anos,anos,brasil
4,íntegra,psl,visita,anos
5,índia,brasil,então,governo
6,ídolo,visita,israel,segundo
7,áreas,então,seguida,afirmou
8,árabe,israel,críticas,disse
9,água,governo,campanha,ser
10,zilhões,afirmou,fazer,sobre


#### 1.2 Top-10 palavras associadas a 'Educação'

In [257]:
table_2

,MIM,EMIM,X²,Dice
1,ávila,vélez,vélez,vélez
2,zero,sabemos,sabemos,sabemos
3,vélez,ricardo,ricardo,ricardo
4,visando,afirmação,afirmação,afirmação
5,vinicius,pasta,soberana,pasta
6,vinculando,completou,saúde,completou
7,vigor,apenas,rodriguez,soberana
8,vidas,seguida,risca,saúde
9,viciado,porque,didáticos,rodriguez
10,verídica,soberana,constitucional,risca


#### 1.3 Top-10 palavras associadas a 'Ministério'

In [258]:
table_3

,MIM,EMIM,X²,Dice
1,ínsita,público,tcu,público
2,zero,tcu,público,diz
3,you,militares,procuradores,militares
4,wti,voz,petrobrás,segundo
5,wikileaks,união,força,sobre
6,whatsapp,diz,conselho,voz
7,web,deste,aplicação,união
8,vítima,tribunal,voz,tribunal
9,véspera,procuradores,união,ainda
10,válidos,petrobrás,militares,deste


#### 1.4 Top-10 palavras associadas a 'Brasil'

In [259]:
table_4

,MIM,EMIM,X²,Dice
1,útil,brasileira,brasileira,bolsonaro
2,única,outros,partir,jair
3,índia,bolsonaro,onde,sobre
4,ávila,jair,resistência,brasileira
5,áreas,onde,luiz,segundo
6,áquila,países,outros,ser
7,zonas,estados,tempo,país
8,zilhões,partir,momento,governo
9,zero,tempo,países,ter
10,you,momento,regime,ainda


#### 1.5 Top-10 palavras associadas a 'Economia'

In [260]:
table_5

,MIM,EMIM,X²,Dice
1,única,guedes,guedes,guedes
2,ávila,proposta,proposta,previdência
3,zero,previdência,previdência,proposta
4,zeca,reforma,cenário,reforma
5,wti,bilhões,reforma,paulo
6,walter,paulo,últimos,ano
7,véspera,últimos,meses,bilhões
8,votada,meses,maiores,ministro
9,volatilidade,maiores,empresas,nesta
10,vocação,empresas,audiência,país


#### De acordo com os resultados, temos 3 métricas com resultados satisfatórios:
* EMIM;
*X²;
*Dice;

Elas apresentam resultados semelhantes e que fazem sentido com o contexto da palavra consultada. Em contrapartida, o MIM tem resultados bem distópicos e isso é causado porque os valores retornado pela métrica é exatamente o mesmo para uma série de palavras e por isso o algoritmo de montagem das tabelas não consegue identificar com precisão qual as palavras que realmente estão próximas da palavra consultada.

Pensando na complexidade de cada métrica, a que na minha opinião se destaca é o Dice. Com cálculo bem simples, ele apresenta bons resultados.

### 2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

#### Algortimo escolhido: Termo-por-vez

Como solução para o problema, montamos o ranking das palavras considerando apenas os top-k documentos para a palavra da consulta. E então, expandindo a consulta a partir desse ranking para todas as palavras no ranking inicial.

In [0]:
def term_a_time(query, indexes, k):    
    a = {} # Accumulator HashTable
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    return heapq.nlargest(k, rank)

In [0]:
def top_docs(query, k):
  
  top_docs = [doc for score,doc in term_a_time(query, indexes, k)]
  top_docs_ranking = {} 
  
  for word in indexes:
    top_docs_ranking[word] = []
    for doc_score in indexes[word]:
      if doc_score[0] in top_docs:
        top_docs_ranking[word].append(doc_score)
        
  return top_docs_ranking

#### Top-3 Documentos

Expansão de consulta para os top-3 documentos das consultas selecionadas inicialmente.

In [0]:
k = 3
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-3 Documentos de 'Bolsonaro'

In [265]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,jair,sobre
2,presidente,presidente
3,brasil,jair
4,anos,feira
5,governo,estado
6,segundo,disse
7,afirmou,afirmou
8,disse,vítimas
9,ser,visita
10,sobre,usando


##### Top-10 palavras associadas aos Top-3 Documentos de 'Educação'

In [266]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,vélez,ser
2,sabemos,porque
3,ricardo,ministro
4,afirmação,governo
5,pasta,contra
6,completou,brasil
7,soberana,época
8,saúde,vélez
9,rodriguez,votação
10,risca,tipo


##### Top-10 palavras associadas aos Top-3 Documentos de 'Ministério'

In [267]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,público,ministro
2,diz,sobre
3,militares,segundo
4,segundo,público
5,sobre,presidente
6,voz,pessoas
7,união,lei
8,tribunal,lava
9,ainda,jato
10,deste,jair


##### Top-10 palavras associadas aos Top-3 Documentos de 'Brasil'

In [268]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,segundo
2,jair,presidente
3,sobre,porque
4,brasileira,pode
5,segundo,país
6,ser,mundo
7,país,meio
8,governo,internacional
9,ter,internacionais
10,ainda,hoje


##### Top-10 palavras associadas aos Top-3 Documentos de 'Economia'

In [269]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,guedes,sobre
2,previdência,política
3,proposta,nacional
4,reforma,meses
5,paulo,governo
6,ano,congresso
7,bilhões,ano
8,ministro,últimos
9,nesta,índice
10,país,vai


#### Top-5 Documentos

Expansão de consulta para os top-5 documentos das consultas selecionadas inicialmente.

In [0]:
k = 5
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-5 Documentos de 'Bolsonaro'

In [272]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,jair,sobre
2,presidente,presidente
3,brasil,jair
4,anos,disse
5,governo,twitter
6,segundo,brasil
7,afirmou,visita
8,disse,segundo
9,ser,oficial
10,sobre,morte


##### Top-10 palavras associadas aos Top-5 Documentos de 'Educação'

In [273]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,vélez,ser
2,sabemos,ministro
3,ricardo,brasil
4,afirmação,sobre
5,pasta,porque
6,completou,jair
7,soberana,governo
8,saúde,contra
9,rodriguez,bolsonaro
10,risca,apenas


##### Top-10 palavras associadas aos Top-5 Documentos de 'Ministério'

In [274]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,público,acordo
2,diz,sobre
3,militares,segundo
4,segundo,presidente
5,sobre,pessoas
6,voz,ministro
7,união,jair
8,tribunal,informações
9,ainda,governo
10,deste,federal


##### Top-10 palavras associadas aos Top-5 Documentos de 'Brasil'

In [275]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,país
2,jair,ter
3,sobre,presidente
4,brasileira,pode
5,segundo,mundo
6,ser,internacional
7,país,hoje
8,governo,bolsonaro
9,ter,unidos
10,ainda,ser


##### Top-10 palavras associadas aos Top-5 Documentos de 'Economia'

In [276]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,guedes,sobre
2,previdência,país
3,proposta,paulo
4,reforma,nesta
5,paulo,nacional
6,ano,ministro
7,bilhões,guedes
8,ministro,governo
9,nesta,estados
10,país,empresas


#### Top-10 Documentos

Expansão de consulta para os top-10 documentos das consultas selecionadas inicialmente.

In [0]:
k = 10
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [0]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-10 Documentos de 'Bolsonaro'

In [279]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,jair,jair
2,presidente,presidente
3,brasil,disse
4,anos,israel
5,governo,feira
6,segundo,brasil
7,afirmou,visita
8,disse,sobre
9,ser,segundo
10,sobre,ministro


##### Top-10 palavras associadas aos Top-10 Documentos de 'Educação'

In [280]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,vélez,ser
2,sabemos,ministro
3,ricardo,brasil
4,afirmação,sobre
5,pasta,porque
6,completou,jair
7,soberana,governo
8,saúde,contra
9,rodriguez,bolsonaro
10,risca,apenas


##### Top-10 palavras associadas aos Top-10 Documentos de 'Ministério'

In [281]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,público,jair
2,diz,diz
3,militares,bolsonaro
4,segundo,sobre
5,sobre,brasil
6,voz,segundo
7,união,público
8,tribunal,presidente
9,ainda,ministro
10,deste,governo


##### Top-10 palavras associadas aos Top-10 Documentos de 'Brasil'

In [282]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,bolsonaro,país
2,jair,ter
3,sobre,sobre
4,brasileira,governo
5,segundo,bolsonaro
6,ser,ainda
7,país,todo
8,governo,ser
9,ter,presidente
10,ainda,jair


##### Top-10 palavras associadas aos Top-10 Documentos de 'Economia'

In [283]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,guedes,ministro
2,previdência,sobre
3,proposta,paulo
4,reforma,guedes
5,paulo,governo
6,ano,reforma
7,bilhões,previdência
8,ministro,país
9,nesta,feira
10,país,quarta


#### Resultado:

Podemos perceber, que ao aumentar o k os resultados tendem a produzir um resultado de maior qualidade, pois assim teremos maior dados sobre a ligação entre as palavras. Quando comparamos o top-3 ao top-10, fica nítido a presença de palavras com maior signifcado e relevância para o contexto que a palavra da consulta está inserida.

Em contrapartida, a adesão de top-k documentos para a expansão da consulta, agiliza bastante a computação para nos mostrar as palavras mais relacionadas àquela consulta.